# 📘 아메리칸 풋옵션 가격 계산


## ✅ 문제  
다음은 풋옵션과 관련된 정보입니다:

- 옵션 유형: **아메리칸 풋옵션**
- 현재 주가: 100  
- 행사가격: 100  
- 무위험이자율: 6%  
- 배당률: 0%  
- 변동성: 20%  
- 만기: 5년  
- 이항트리 스텝 수: 200  
- 시간 간격: $dt = \frac{T}{N} = \frac{5}{200}$

> 풋옵션의 가격을 소수점 **네 번째 자리까지 반올림**하여 구하시오.  
> 예: `0.12345 → 0.1235`  
>  
> 또한 소수점 처리를 위해 코드 상단에  
> `np.set_printoptions(precision=4, suppress=True)`  
> 를 사용하시오.


## 🧮 해설



### 1. 이항 트리(Binomial Tree) 개념  
이 모델은 옵션 만기까지 주가가 매 스텝마다 오르거나 내리는 이산적 구조를 따릅니다.  
각 노드마다 주가 및 옵션 가치를 계산하고, 거꾸로(backward) 올라가며 현재 가치를 구합니다.

---

### 2. 사용되는 수식

- 상승률:  
  $u = e^{\sigma \sqrt{dt}}$

- 하락률:  
  $d = \frac{1}{u}$

- 리스크 중립 확률:  
  $p = \frac{e^{(r - q) dt} - d}{u - d}$

- 할인율:  
  $\text{discount} = e^{-r dt}$

---

### 3. 아메리칸 풋옵션의 특징  
- 유럽형 옵션과 달리, 만기 이전에도 **조기행사 가능**
- 따라서 각 시점에서의 옵션 가치는 다음 중 **더 큰 값**으로 결정됨:  
  - 보유 가치 (기대 현재가치)  
  - 즉시 행사 가치


In [ ]:
import numpy as np

# 소수점 설정
np.set_printoptions(precision=4, suppress=True)

# 파라미터 설정
S0 = 100         # 현재 주가
K = 100          # 행사가격 (Strike Price)
r = 0.06         # 무위험이자율
q = 0.0          # 배당률 (Dividend yield, quotient에서 약자를 따옴)
sigma = 0.20     # 변동성
T = 5            # 만기
N = 200          # 스텝 수

# 시간 간격
dt = T / N
u = np.exp(sigma * np.sqrt(dt))        # 상승 비율
d = 1 / u                              # 하락 비율
p = (np.exp((r - q) * dt) - d) / (u - d)  # 리스크 중립 확률
discount = np.exp(-r * dt)             # 할인율

# 주가 트리 구성
stock_tree = np.zeros((N + 1, N + 1))
for i in range(N + 1):
    for j in range(i + 1):
        stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

# 옵션 가치 트리 구성 (아메리칸 풋옵션)
option_tree = np.zeros((N + 1, N + 1))
# 만기 시점의 풋옵션 가치
option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)

# 백워드 루핑으로 옵션 가격 계산
for i in range(N - 1, -1, -1):
    for j in range(i + 1):
        hold = discount * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])
        exercise = K - stock_tree[j, i]
        option_tree[j, i] = max(hold, exercise)  # 아메리칸 옵션이므로 조기행사 고려

# 결과
option_price = option_tree[0, 0]
option_price = round(option_price, 4)
option_price


np.float64(8.9765)

In [ ]:
import numpy as np

np.set_printoptions(precision = 4, suppress = True)

# 파라미터 설정
S0 = 100; K = 90; r = 0.05; q = 0; T = 3; sigma = 0.3; N = 300; T = 3

# Binomial Tree 공식 정리
dt = T/N;
u = np.exp(sigma * np.sqrt(dt)) # u = e^(σ√dt)
d = 1 / u
p = (np.exp((r - q) * dt) - d) / (u - d) # 리스크 중립확률 p = (e^{(r - q)dt} - d) / (u - d)
discount = np.exp(-r * dt) # discount = e^{-r dt}